##### Load libs:

In [1]:
import pandas as pd
import sqlite3

##### 1.Create connect:

In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

##### 2.Take schema for table "test":

In [3]:
print(pd.io.sql.read_sql("PRAGMA table_info(test)",connect))

   cid             name  type  notnull dflt_value  pk
0    0              uid  TEXT        0       None   0
1    1          labname  TEXT        0       None   0
2    2  first_commit_ts  TEXT        0       None   0
3    3    first_view_ts  TEXT        0       None   0


##### 3.First 10 string table "test":

In [4]:
print(pd.io.sql.read_sql("SELECT * FROM test LIMIT 10;",connect))

       uid   labname             first_commit_ts               first_view_ts
0   user_1    laba04  2020-04-26 17:06:18.462708  2020-04-26 21:53:59.624136
1   user_1   laba04s  2020-04-26 17:12:11.843671  2020-04-26 21:53:59.624136
2   user_1    laba05  2020-05-02 19:15:18.540185  2020-04-26 21:53:59.624136
3   user_1    laba06  2020-05-17 16:26:35.268534  2020-04-26 21:53:59.624136
4   user_1   laba06s  2020-05-20 12:23:37.289724  2020-04-26 21:53:59.624136
5   user_1  project1  2020-05-14 20:56:08.898880  2020-04-26 21:53:59.624136
6  user_10    laba04  2020-04-25 08:24:52.696624  2020-04-18 12:19:50.182714
7  user_10   laba04s  2020-04-25 08:37:54.604222  2020-04-18 12:19:50.182714
8  user_10    laba05  2020-05-01 19:27:26.063245  2020-04-18 12:19:50.182714
9  user_10    laba06  2020-05-19 11:39:28.885637  2020-04-18 12:19:50.182714


##### 4.Min delta:

In [34]:
df_min=pd.io.sql.read_sql(
	"SELECT "
		"t.uid, " \
		"MIN(" \
		""#ABS?
		"(julianday(t.first_commit_ts)-julianday(datetime(d.deadlines, 'unixepoch'))))*24 AS min_diff_deadline_hour " \
	"FROM test t " \
	"JOIN deadlines d ON t.labname=d.labs " \
	"WHERE t.labname != 'project1' " \
	"GROUP BY t.uid;", 
	connect
)
print(df_min)

        uid  min_diff_deadline_hour
0    user_1             -175.556592
1   user_10             -132.341698
2   user_14             -200.766302
3   user_17              -81.591403
4   user_18              -10.973375
5   user_19             -148.916028
6   user_21             -126.199587
7   user_25             -150.869725
8   user_28             -174.852984
9    user_3             -182.055144
10  user_30             -202.384730


##### 5.Max delta

In [48]:
df_max=pd.io.sql.read_sql(
	"SELECT "
		"t.uid, " \
		"MAX(" \
		""#ABS?
		"(julianday(t.first_commit_ts)-julianday(datetime(d.deadlines, 'unixepoch'))))*24 AS max_diff_deadline_hour " \
	"FROM test t " \
	"JOIN deadlines d ON t.labname=d.labs " \
	"WHERE t.labname != 'project1' " \
	"GROUP BY t.uid;", 
	connect
)
print(df_max)

        uid  max_diff_deadline_hour
0    user_1               -6.796432
1   user_10              -39.367888
2   user_14              -84.448466
3   user_17              -34.643043
4   user_18               -3.933907
5   user_19              -32.729282
6   user_21              -33.905274
7   user_25               -2.867236
8   user_28               -8.103915
9    user_3              -60.511392
10  user_30              -52.477951


##### 6.Avg delta:

In [49]:
df_avg=pd.io.sql.read_sql(
	"SELECT "
		"AVG(" \
		""#ABS?
		"(julianday(t.first_commit_ts)-julianday(datetime(d.deadlines, 'unixepoch'))))*24 AS avg_diff_deadline_hour " \
	"FROM test t " \
	"JOIN deadlines d ON t.labname=d.labs " \
	"WHERE t.labname != 'project1' " \
	"GROUP BY t.uid;", 
	connect
)
print(df_avg)

    avg_diff_deadline_hour
0               -65.119644
1               -75.242310
2              -159.568696
3               -62.207513
4                -6.367907
5               -99.440298
6               -96.111041
7               -93.474751
8               -86.793652
9              -105.738041
10             -145.528546


##### 7.Correlation:

In [68]:
views_diff=pd.io.sql.read_sql(
	"SELECT " \
		"t.uid, " \
		"AVG(" \
		""#ABS?
		"(julianday(t.first_commit_ts)-julianday(datetime(d.deadlines, 'unixepoch'))))*24 AS avg_diff, " \
		"COUNT(*) AS pageviews " \
	"FROM test t " \
	"JOIN deadlines d ON t.labname=d.labs " \
	"WHERE t.labname != 'project1' " \
	"GROUP BY t.uid;", 
	connect
)
print(views_diff)

        uid    avg_diff  pageviews
0    user_1  -65.119644          5
1   user_10  -75.242310          5
2   user_14 -159.568696          3
3   user_17  -62.207513          5
4   user_18   -6.367907          3
5   user_19  -99.440298          4
6   user_21  -96.111041          4
7   user_25  -93.474751          5
8   user_28  -86.793652          5
9    user_3 -105.738041          5
10  user_30 -145.528546          4


In [69]:
print(views_diff['avg_diff'].corr(views_diff['pageviews']))

0.11837428457444202


##### Close connect:

In [84]:
connect.close()

### CHECK:

In [83]:
print(
	pd.io.sql.read_sql(
		"SELECT "
			"uid, " \
			"labname, "
			"(julianday(first_commit_ts)-julianday(datetime(d.deadlines, 'unixepoch')))*24 AS diff " \
		"FROM test t " \
		"JOIN deadlines d ON d.labs=t.labname " \
		"WHERE uid = 'user_1' AND labname != 'project1' "
		,connect
	)
)

      uid  labname        diff
0  user_1   laba04   -6.894594
1  user_1  laba04s   -6.796432
2  user_1   laba05  -28.744572
3  user_1   laba06 -175.556592
4  user_1  laba06s -107.606031
